In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\Projects\\AI Projects\\Text-Summarizer-Application\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\Projects\\AI Projects\\Text-Summarizer-Application'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarizer.utils.common import get_size
from textSummarizer.logging import logger

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def dowload_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
        logger.info(f"File extracted at location: {unzip_path}")


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.dowload_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-05 12:33:32,242: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-05 12:33:32,244: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-05 12:33:32,245: INFO: common: Directory created at: artifacts]
[2026-01-05 12:33:32,246: INFO: common: Directory created at: artifacts/data_ingestion]
[2026-01-05 12:33:34,027: INFO: 2670222456: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B434:2119BE:1899A8B:3226E75:695B6244
Accept-Ranges: bytes
Date: Mon, 05 Jan 2026 07:03:33 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210063-BOM